# 1. 万向锁发生的条件

## 1.1 XYZ型单轴运动

### 1. 万向锁（Gimbal Lock）由“中间轴”决定

万向锁只与旋转序列中的第 2 个轴（中间轴）有关。

在 XYZ 顺序中，中间轴是 Y。当 $Y = \pm 90^\circ$ 时死锁。

在 YXZ 顺序中，中间轴是 X。当 $X = \pm 90^\circ$ 时死锁（实验数据证明：输入 X=90° 时 Lock=YES）。

第 1 轴和第 3 轴（首尾轴）永远是安全的，无论它们转多少度，都不会主动触发万向锁。

### 2. 定义域的“半圆限制”也仅针对“中间轴”

求解器强制将序列中的第 2 个轴（中间轴）限制在 $[-90^\circ, 90^\circ]$ 之间。

证据 A (XYZ 实验)：输入 $Y=91^\circ$ $\to$ 结果 $Y$ 被压回 $89^\circ$。

证据 B (YXZ 实验)：

输入 $Y=91^\circ$（此时 Y 是第 1 轴） $\to$ 结果 $Y=91^\circ$。（第 1 轴享有全圆自由度！）

输入 $X=91^\circ$（此时 X 是第 2 轴） $\to$ 结果 $X$ 被压回 $89^\circ$。（第 2 轴受到半圆限制！）

### 3. 别名现象（Aliasing）的触发机制

当物理动作试图让“中间轴”超出 $\pm 90^\circ$ 范围时，数学上会发生“折叠（Folding）”或“别名”现象。

系统会通过同时翻转第 1 轴和第 3 轴各 $180^\circ$，将中间轴的角度“折”回安全区（$[-90, 90]$）。

例子 (YXZ)：输入 $X=180^\circ$（中间轴完全翻转）。求解器为了避免 $X=180$，给出的解是 Y=180, X=0, Z=180。它宁愿让首尾两个轴都转半圈，也要把中间轴归零。

In [2]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def get_rotation_matrix(axis, degrees):
    """ 生成基础旋转矩阵 """
    rad = np.radians(degrees)
    c, s = np.cos(rad), np.sin(rad)
    if axis == 'x':
        return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])
    elif axis == 'y':
        return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])
    elif axis == 'z':
        return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])

def check_gimbal_lock(euler_angles, degrees=True):
    """ 
    判定是否发生万向锁
    对于 XYZ 顺序，中间轴是 Y (索引1)。
    如果 Y 轴角度接近 90 或 270 (-90)，则处于死锁状态。
    """
    y_angle = euler_angles[1]
    # 将角度归一化到 [-180, 180] 方便判断，或者直接判断模 180
    # 简单起见，直接判断是否接近 90 或 270
    abs_y = abs(y_angle)
    is_lock = np.isclose(abs_y, 90.0) or np.isclose(abs_y, 270.0)
    return is_lock

def analyze_single_axis_rotation():
    axes = ['x', 'y', 'z']
    angles = [0, 1, 90, 91, 180, 181, 270, 271]
    
    print(f"{'Input':<15} | {'Solver':<8} | {'Result (Angle 0, 1, 2)':<35} | {'Gimbal Lock?'}")
    print("-" * 85)

    for axis in axes:
        for ang in angles:
            # 1. 生成旋转矩阵
            mat = get_rotation_matrix(axis, ang)
            r = R.from_matrix(mat)
            
            # 2. 求解欧拉角
            # 方案 A: 内旋 XYZ
            euler_XYZ = r.as_euler('XYZ', degrees=True)
            lock_XYZ = check_gimbal_lock(euler_XYZ)
            
            # 方案 B: 外旋 xyz
            euler_xyz = r.as_euler('xyz', degrees=True)
            lock_xyz = check_gimbal_lock(euler_xyz) # 外旋 xyz 的中间轴也是 y
            
            # 3. 打印结果
            input_str = f"{axis.upper()}={ang}°"
            
            # 打印 XYZ 结果
            val_str_XYZ = f"[{euler_XYZ[0]:.1f}, {euler_XYZ[1]:.1f}, {euler_XYZ[2]:.1f}]"
            lock_str_XYZ = "YES (Danger)" if lock_XYZ else "No"
            print(f"{input_str:<15} | {'XYZ':<8} | {val_str_XYZ:<35} | {lock_str_XYZ}")
            
            # 打印 xyz 结果 (仅在结果不同时强调，通常单轴旋转结果相同)
            val_str_xyz = f"[{euler_xyz[0]:.1f}, {euler_xyz[1]:.1f}, {euler_xyz[2]:.1f}]"
            lock_str_xyz = "YES (Danger)" if lock_xyz else "No"
            # print(f"{'':<15} | {'xyz':<8} | {val_str_xyz:<35} | {lock_str_xyz}")
            
            if not np.allclose(euler_XYZ, euler_xyz):
                 print(f"{'':<15} | {'xyz':<8} | {val_str_xyz:<35} | {lock_str_xyz} <--- DIFF")
            
        print("-" * 85) # 每个轴结束分割线

# 运行分析
analyze_single_axis_rotation()

Input           | Solver   | Result (Angle 0, 1, 2)              | Gimbal Lock?
-------------------------------------------------------------------------------------
X=0°            | XYZ      | [0.0, 0.0, 0.0]                     | No
X=1°            | XYZ      | [1.0, 0.0, 0.0]                     | No
X=90°           | XYZ      | [90.0, 0.0, 0.0]                    | No
X=91°           | XYZ      | [91.0, 0.0, 0.0]                    | No
X=180°          | XYZ      | [180.0, 0.0, 0.0]                   | No
X=181°          | XYZ      | [-179.0, 0.0, 0.0]                  | No
X=270°          | XYZ      | [-90.0, 0.0, 0.0]                   | No
X=271°          | XYZ      | [-89.0, 0.0, 0.0]                   | No
-------------------------------------------------------------------------------------
Y=0°            | XYZ      | [0.0, 0.0, 0.0]                     | No
Y=1°            | XYZ      | [0.0, 1.0, 0.0]                     | No
Y=90°           | XYZ      | [0.0, 90.0, 0.0]   

/tmp/ipykernel_15862/1751847779.py:43: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  euler_XYZ = r.as_euler('XYZ', degrees=True)
/tmp/ipykernel_15862/1751847779.py:47: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  euler_xyz = r.as_euler('xyz', degrees=True)


将中间轴换成X轴，发现也有90度保护，证明Y轴没有特殊性。

In [3]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def get_rotation_matrix(axis, degrees):
    """ 生成基础旋转矩阵 """
    rad = np.radians(degrees)
    c, s = np.cos(rad), np.sin(rad)
    if axis == 'x':
        return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])
    elif axis == 'y':
        return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])
    elif axis == 'z':
        return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])

def check_gimbal_lock_middle(euler_angles):
    """ 
    通用判定：检查序列中的'中间轴'（索引1）是否死锁
    """
    middle_angle = euler_angles[1]
    abs_ang = abs(middle_angle)
    # 判定是否接近 90 或 270
    return np.isclose(abs_ang, 90.0) or np.isclose(abs_ang, 270.0)

def analyze_YXZ_order():
    print(f"\n{'='*20} 测试序列: YXZ (中间轴为 X) {'='*20}")
    
    axes = ['x', 'y', 'z']
    # 重点关注 90, 91 附近的跳变
    angles = [0, 90, 91, 180] 
    
    # 注意表头顺序：根据 YXZ 序列，输出分别是 Angle Y, Angle X, Angle Z
    print(f"{'Input':<15} | {'Solver':<8} | {'Result [Y, X, Z]':<35} | {'Lock (Mid=X)?'}")
    print("-" * 85)

    for axis in axes:
        for ang in angles:
            # 1. 生成旋转矩阵
            mat = get_rotation_matrix(axis, ang)
            r = R.from_matrix(mat)
            
            # 2. 求解欧拉角 (序列 YXZ)
            # 返回值的顺序对应字符串顺序：result[0]=Y, result[1]=X, result[2]=Z
            euler_YXZ = r.as_euler('YXZ', degrees=True)
            
            # 3. 检查中间轴 (索引1，即 X) 是否死锁
            is_lock = check_gimbal_lock_middle(euler_YXZ)
            
            # 4. 打印
            input_str = f"{axis.upper()}={ang}°"
            val_str = f"[{euler_YXZ[0]:.1f}, {euler_YXZ[1]:.1f}, {euler_YXZ[2]:.1f}]"
            
            # 只有当发生死锁或结果数值看起来“奇怪”（即输入轴与输出轴不匹配）时，才标记
            lock_str = "YES (X=90)" if is_lock else "No"
            
            # 辅助标记：如果发生了翻转 (例如输入 91 输出 89)
            note = ""
            if axis == 'x' and ang == 91 and np.isclose(euler_YXZ[1], 89.0):
                note = "<-- 限制生效! X被压回89"
            
            print(f"{input_str:<15} | {'YXZ':<8} | {val_str:<35} | {lock_str} {note}")
        print("-" * 85)

analyze_YXZ_order()


==================== 测试序列: YXZ (中间轴为 X) ====================
Input           | Solver   | Result [Y, X, Z]                    | Lock (Mid=X)?
-------------------------------------------------------------------------------------
X=0°            | YXZ      | [0.0, 0.0, 0.0]                     | No 
X=90°           | YXZ      | [0.0, 90.0, 0.0]                    | YES (X=90) 
X=91°           | YXZ      | [180.0, 89.0, 180.0]                | No <-- 限制生效! X被压回89
X=180°          | YXZ      | [180.0, 0.0, 180.0]                 | No 
-------------------------------------------------------------------------------------
Y=0°            | YXZ      | [0.0, 0.0, 0.0]                     | No 
Y=90°           | YXZ      | [90.0, 0.0, 0.0]                    | No 
Y=91°           | YXZ      | [91.0, 0.0, 0.0]                    | No 
Y=180°          | YXZ      | [180.0, 0.0, 0.0]                   | No 
-------------------------------------------------------------------------------------
Z=0°   

/tmp/ipykernel_15862/2178685320.py:43: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  euler_YXZ = r.as_euler('YXZ', degrees=True)


## 1.2 双轴旋转

内旋反解为XYZ，万向锁产生的条件为下面二选一:

1.涉及到Y轴的90度或-90度旋转。

2.ZX都是90度的旋转。

In [5]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def get_rotation_matrix(axis, degrees):
    """ 生成基础旋转矩阵 """
    rad = np.radians(degrees)
    c, s = np.cos(rad), np.sin(rad)
    if axis == 'x':
        return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])
    elif axis == 'y':
        return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])
    elif axis == 'z':
        return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])

def check_lock_xyz(euler):
    """ 
    对于 XYZ 顺序，中间轴是 Y (索引1)
    判定死锁条件：Y 接近 90 或 270
    """
    y_ang = abs(euler[1])
    return np.isclose(y_ang, 90.0) or np.isclose(y_ang, 270.0)

def analyze_intrinsic_only():
    # 分组测试
    # 组1: 安全组 (不含 Y)
    group_safe = [('x', 'z'), ('z', 'x')]
    # 组2: 危险组 (含 Y)
    group_danger = [('x', 'y'), ('y', 'x'), ('z', 'y'), ('y', 'z')]
    
    test_angles = [0, 90, 91]
    
    print(f"{'Input Seq':<10} | {'Angles':<10} | {'Solver':<6} | {'Result [X, Y, Z]':<32} | {'Lock (Y=90)?'}")
    print("=" * 90)

    # ==========================
    # Part 1: 安全组测试 (XZ, ZX)
    # ==========================
    print(">>> Part 1: 安全组 (不涉及中间轴 Y 的旋转)")
    for axis1, axis2 in group_safe:
        for ang1 in test_angles:
            for ang2 in test_angles:
                # 过滤掉全0 (无意义)
                if ang1 == 0 and ang2 == 0: continue
                
                # 生成矩阵 (内旋: R1 @ R2)
                R1 = get_rotation_matrix(axis1, ang1)
                R2 = get_rotation_matrix(axis2, ang2)
                M = R1 @ R2
                
                # 求解
                r = R.from_matrix(M)
                euler = r.as_euler('XYZ', degrees=True)
                is_lock = check_lock_xyz(euler)
                
                # 打印
                input_str = f"{axis1.upper()}->{axis2.upper()}"
                ang_str = f"{ang1}, {ang2}"
                res_str = f"[{euler[0]:.1f}, {euler[1]:.1f}, {euler[2]:.1f}]"
                lock_str = "YES" if is_lock else "No"
                
                print(f"{input_str:<10} | {ang_str:<10} | {'XYZ':<6} | {res_str:<32} | {lock_str}")
    
    print("-" * 90)
    
    # ==========================
    # Part 2: 危险组测试 (含 Y)
    # ==========================
    print(">>> Part 2: 危险组 (涉及中间轴 Y 的旋转)")
    for axis1, axis2 in group_danger:
        for ang1 in test_angles:
            for ang2 in test_angles:
                # 过滤掉全0
                if ang1 == 0 and ang2 == 0: continue
                # 过滤掉简单的非90/91组合，专注于问题点
                if 90 not in [ang1, ang2] and 91 not in [ang1, ang2]: continue

                # 生成矩阵
                R1 = get_rotation_matrix(axis1, ang1)
                R2 = get_rotation_matrix(axis2, ang2)
                M = R1 @ R2
                
                # 求解
                r = R.from_matrix(M)
                euler = r.as_euler('XYZ', degrees=True)
                is_lock = check_lock_xyz(euler)
                
                # 判定是否有翻转/折叠现象 (Y 被压回 90 以内)
                # 逻辑：如果我们输入的 Y 是 91，但解出来的 Y 是 89，说明发生了折叠
                is_folded = False
                target_y_in_input = 0
                if axis1 == 'y': target_y_in_input = ang1
                if axis2 == 'y': target_y_in_input = ang2
                
                if target_y_in_input == 91 and np.isclose(euler[1], 89.0):
                    is_folded = True
                
                # 打印
                input_str = f"{axis1.upper()}->{axis2.upper()}"
                ang_str = f"{ang1}, {ang2}"
                res_str = f"[{euler[0]:.1f}, {euler[1]:.1f}, {euler[2]:.1f}]"
                lock_str = "YES" if is_lock else "No"
                
                note = ""
                if is_folded: note = "<-- Aliasing/Folded!"
                
                print(f"{input_str:<10} | {ang_str:<10} | {'XYZ':<6} | {res_str:<32} | {lock_str} {note}")

analyze_intrinsic_only()

Input Seq  | Angles     | Solver | Result [X, Y, Z]                 | Lock (Y=90)?
>>> Part 1: 安全组 (不涉及中间轴 Y 的旋转)
X->Z       | 0, 90      | XYZ    | [0.0, 0.0, 90.0]                 | No
X->Z       | 0, 91      | XYZ    | [0.0, 0.0, 91.0]                 | No
X->Z       | 90, 0      | XYZ    | [90.0, 0.0, 0.0]                 | No
X->Z       | 90, 90     | XYZ    | [90.0, 0.0, 90.0]                | No
X->Z       | 90, 91     | XYZ    | [90.0, 0.0, 91.0]                | No
X->Z       | 91, 0      | XYZ    | [91.0, 0.0, 0.0]                 | No
X->Z       | 91, 90     | XYZ    | [91.0, 0.0, 90.0]                | No
X->Z       | 91, 91     | XYZ    | [91.0, 0.0, 91.0]                | No
Z->X       | 0, 90      | XYZ    | [90.0, 0.0, 0.0]                 | No
Z->X       | 0, 91      | XYZ    | [91.0, 0.0, 0.0]                 | No
Z->X       | 90, 0      | XYZ    | [0.0, 0.0, 90.0]                 | No
Z->X       | 90, 90     | XYZ    | [90.0, 90.0, 0.0]                | YES
Z->X     

/tmp/ipykernel_15862/912339282.py:52: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  euler = r.as_euler('XYZ', degrees=True)
/tmp/ipykernel_15862/912339282.py:84: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  euler = r.as_euler('XYZ', degrees=True)


# 2. 欧拉角求解顺序的影响

## 2.1 XYZ型内旋

在非极端条件，求解顺序会导致绕对应轴的角度有显著不同。

In [9]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def get_rotation_matrix(axis, degrees):
    """ 生成基础旋转矩阵 """
    rad = np.radians(degrees)
    c, s = np.cos(rad), np.sin(rad)
    if axis == 'x':
        return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])
    elif axis == 'y':
        return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])
    elif axis == 'z':
        return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])

def analyze_order_influence():
    # 1. 生成目标姿态 (Ground Truth)
    # 设定基准: 内旋 ZYX (Yaw=50, Pitch=40, Roll=30)
    # 数学构建: 内旋 Z->Y->X 等价于矩阵乘法 R = R_z @ R_y @ R_x
    target_z = 50
    target_y = 40
    target_x = 30
    
    R_z = get_rotation_matrix('z', target_z)
    R_y = get_rotation_matrix('y', target_y)
    R_x = get_rotation_matrix('x', target_x)
    
    # 合成旋转矩阵 M
    M_target = R_z @ R_y @ R_x
    
    # 创建 Scipy 旋转对象
    r_obj = R.from_matrix(M_target)
    
    print(f"{'='*20} 目标姿态 (Ground Truth) {'='*20}")
    print(f"生成逻辑: 内旋 ZYX (Z={target_z}, Y={target_y}, X={target_x})")
    print(f"(这是一个固定的物理姿态)\n")

    # 2. 定义所有内旋求解顺序
    orders = ['XYZ', 'XZY', 'YXZ', 'YZX', 'ZXY', 'ZYX']
    
    print(f"{'Solver Seq':<10} | {'Angle X (Roll)':<15} | {'Angle Y (Pitch)':<15} | {'Angle Z (Yaw)':<15}")
    print("-" * 65)

    for seq in orders:
        # 求解欧拉角 (使用大写字母表示内旋)
        angles = r_obj.as_euler(seq, degrees=True)
        
        # 3. 智能映射：将结果数组映射回物理轴以便对比
        # as_euler 返回值的顺序对应 seq 字符串的顺序
        # 例如 seq='YXZ', angles=[val_y, val_x, val_z]
        val_map = {}
        for i, axis_char in enumerate(seq):
            val_map[axis_char] = angles[i]
        
        # 提取各轴数值
        x_val = val_map['X']
        y_val = val_map['Y']
        z_val = val_map['Z']
        
        # 标记是否与原始真值 (X=30) 相等
        mark_x = " (Target)" if np.isclose(x_val, 30.0) else ""
        mark_y = " (Target)" if np.isclose(y_val, 40.0) else ""
        mark_z = " (Target)" if np.isclose(z_val, 50.0) else ""
        
        # 为了排版整齐，如果数值差异大，就不打标记了，只在 ZYX 处打标记验证
        # 这里为了直观，我们只在完全匹配的那一行打星号
        
        # 打印
        print(f"{seq:<10} | {x_val:>9.4f}{mark_x:<9} | {y_val:>9.4f}{mark_y:<9} | {z_val:>9.4f}{mark_z:<9}")

# 运行分析
analyze_order_influence()

==================== 目标姿态 (Ground Truth) ====================
生成逻辑: 内旋 ZYX (Z=50, Y=40, X=30)
(这是一个固定的物理姿态)

Solver Seq | Angle X (Roll)  | Angle Y (Pitch) | Angle Z (Yaw)  
-----------------------------------------------------------------
XYZ        |   -8.9971          |   47.8033          |   42.8535         
XZY        |   25.5042          |   56.3899          |   27.1825         
YXZ        |   -6.0295          |   48.1560          |   36.1633         
YZX        |   -7.4537          |   52.5463          |   35.9320         
ZXY        |   22.5210          |   44.0953          |   29.6394         
ZYX        |   30.0000 (Target) |   40.0000 (Target) |   50.0000 (Target)
